In [42]:
import json
import elasticsearch
from elasticsearch.helpers import bulk

In [43]:
client = elasticsearch.Elasticsearch('localhost:9200')

client.info()

{'name': 'LAPTOP-HOFG0F6S',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': '6GIazPz8RWW2xepk3AnbSA',
 'version': {'number': '7.9.3',
  'build_flavor': 'unknown',
  'build_type': 'unknown',
  'build_hash': 'c4138e51121ef06a6404866cddc601906fe5c868',
  'build_date': '2020-10-16T10:36:16.141335Z',
  'build_snapshot': False,
  'lucene_version': '8.6.2',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [44]:
INDEX_NAME = 'tweets2021'

In [45]:
INDEX_MAPPING = {
    "settings": {
        "number_of_shards": 3,
        "analysis": {
            "normalizer": {
                "hashtag_normalizer": {
                    "filter": [
                        "lowercase",
                        "asciifolding"
                    ],
                    "type": "custom",
                    "char_filter": []
                }
            },
            "analyzer": {
                "tweet_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": ["asciifolding", "lowercase", "stop_pt",
                                "stop_custom", "porter_stem"]
                }
            },
            "filter": {
                "stop_pt": {
                    "type": "stop",
                    "stopwords": "_portuguese_",
                },
                "stop_custom": {
                    "type": "stop",

                    "stopwords": [
                        'nao', 'vc', 'pq', 'ai', 'q', 'ta', 'http', 'pra', 't.co', 'sao', 'to', 'voce', 'so'
                    ]
                }
            }
        }
    },
    "mappings": {
        "dynamic": False,
        "properties": {
            "cashtags": {
                "type": "keyword",
                "normalizer": "hashtag_normalizer"
            },
            "conversation_id": {
                "type": "long"
            },
            "created_at": {
                "type": "text"
            },
            "date": {
                "type": "date",
                "format": "yyyy-MM-dd HH:mm:ss"
            },
            "day": {
                "type": "integer"
            },
            "essid": {
                "type": "keyword"
            },
            "geo_near": {
                "type": "geo_point"
            },
            "geo_tweet": {
                "type": "geo_point"
            },
            "hashtags": {
                "type": "keyword",
                "normalizer": "hashtag_normalizer"
            },
            "hour": {
                "type": "integer"
            },
            "id": {
                "type": "long"
            },
            "lang": {
                "type": "keyword"
            },
            "language": {
                "type": "text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            },
            "link": {
                "type": "text"
            },
            "location": {
                "type": "keyword"
            },
            "mentions": {
                "type": "nested",
                "properties": {
                    "id": {
                        "type": "long"
                    },
                    "name": {
                        "type": "text"
                    },
                    "screen_name": {
                        "type": "text"
                    }
                }
            },
            "name": {
                "type": "text"
            },
            "near": {
                "type": "text"
            },
            "nlikes": {
                "type": "integer"
            },
            "nreplies": {
                "type": "integer"
            },
            "nretweets": {
                "type": "integer"
            },
            "photos": {
                "type": "text"
            },
            "profile_image_url": {
                "type": "text"
            },
            "quote_url": {
                "type": "text"
            },
            "reply_to": {
                "type": "nested",
                "properties": {
                    "id": {
                        "type": "text",
                        "fields": {
                            "keyword": {
                                "type": "keyword",
                                "ignore_above": 256
                            }
                        }
                    },
                    "name": {
                        "type": "text",
                        "fields": {
                            "keyword": {
                                "type": "keyword",
                                "ignore_above": 256
                            }
                        }
                    },
                    "screen_name": {
                        "type": "text",
                        "fields": {
                            "keyword": {
                                "type": "keyword",
                                "ignore_above": 256
                            }
                        }
                    },
                    "user_id": {
                        "type": "keyword"
                    },
                    "username": {
                        "type": "keyword"
                    }
                }
            },
            "retweet": {
                "type": "text"
            },
            "retweet_date": {
                "type": "date",
                "format": "yyyy-MM-dd HH:mm:ss",
                "ignore_malformed": True
            },
            "retweet_id": {
                "type": "keyword"
            },
            "search": {
                "type": "text"
            },
            "source": {
                "type": "keyword"
            },
            "thumbnail": {
                "type": "text"
            },
            "timezone": {
                "type": "keyword"
            },
            "trans_dest": {
                "type": "keyword"
            },
            "trans_src": {
                "type": "keyword"
            },
            "translate": {
                "type": "text"
            },
            "tweet": {
                "type": "text",
                "analyzer": "tweet_analyzer",
                "fielddata": True,
                "fielddata_frequency_filter": {
                    "min": 0.01,
                    "min_segment_size": 10,
                }
            },
            "urls": {
                "type": "keyword"
            },
            "user_id_str": {
                "type": "keyword"
            },
            "user_rt": {
                "type": "keyword"
            },
            "user_rt_id": {
                "type": "keyword"
            },
            "username": {
                "type": "keyword",
                "normalizer": "hashtag_normalizer"
            },
            "video": {
                "type": "integer"
            }
        }
    }
}

In [46]:
# Apagar caso o indice exista
if client.indices.exists(INDEX_NAME):
    client.indices.delete(INDEX_NAME)


ConnectionTimeout: ConnectionTimeout caused by - ReadTimeoutError(HTTPConnectionPool(host='localhost', port=9200): Read timed out. (read timeout=10))

In [ ]:
client.indices.create(INDEX_NAME, body=INDEX_MAPPING)